In [19]:
# Globals
CROP_SIZE = 400
SPREADSHEET = '/vol/research/mammo2/will/data/batches/metadata/3/batch_3_IMAGE.xls'
DICOM_FILES = '/vol/research/mammo2/will/data/batches/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/3/'

In [14]:
# functions
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np
import os
from fnmatch import fnmatch

# File walk method
def getFiles(verbose=1):
    fileList = []
    for path, subdirs, files in os.walk(DICOM_FILES):
        for name in files:
            if fnmatch(name, '*.dcm'):
                fileList.append(os.path.join(path, name))

    print(len(fileList), ' dicom images found')


    # Load dicom files into np array
    dicomImg = np.array([])
    count = 0
    print(len(fileList), ' Files found')
    print('Loading images...')
    for f in fileList:
        dicomImg = np.append(dicomImg, pydicom.dcmread(f))
        count += 1
        print(count, '/', len(fileList))
    print('name:\n', dicomImg[0].PresentationIntentType)
    return dicomImg, fileList




def deletePreProcessed(dicomImg, fileList):
    forPresentationCount = 0
    toDelete = []
    for index, _ in enumerate(dicomImg):
        tmp = _.PresentationIntentType
        if tmp == 'FOR PRESENTATION':
            forPresentationCount += 1
        else:
            toDelete.append(index)
    print('dicomImg len:', len(dicomImg))
    np.delete(dicomImg, toDelete)
    mask = np.ones(len(dicomImg), dtype=bool)
    mask[toDelete] = False
    dicomImg = dicomImg[mask]
    fileList = np.asarray(fileList)
    fileList = fileList[mask]
    print('\nTotal DICOM: ', len(dicomImg))
    print('Total FOR PROCESSING: ', forPresentationCount)
    return dicomImg, fileList

# Get the file names/path of the contralateral images
def getContralateral(fileList):


# Remove files that are not in the spreadsheet
def getFileListROI(filesList):
    xls = pd.ExcelFile(SPREADSHEET)
    sheet = xls.parse(1)
    
    img = {}
    filesListROI = []
    for i in range(len(fileList)):
        key = os.path.basename(fileList[i])[:-4]
        # Find row in the xml file that holds the img info
        try:
            indx = [_==key for _ in sheet['ImageSOPIUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
            filesListROI.append(fileList[i])            
        except ValueError:
            print('MY_ERROR: key not found:\n', key)
    return filesListROI

   
# Import xls file, extract ROI coords, get pixel array from DICOM image
def buildDict(dicomImg, fileList):
    #xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/download/batch_1_IMAGE.xls')
    #xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch_50_IMAGE.xls')
    xls = pd.ExcelFile(SPREADSHEET)
    sheet = xls.parse(0)

    # Create a dict where the key is the image name
    # Each key has the image, and coords
    img = {}
    for i in range(len(fileList)):
        key = os.path.basename(fileList[i])[:-4]
        # Find row in the xml file that holds the img info
        try:
            indx = [_==key for _ in sheet['ImageSOPIUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
            img.update({key:{}})
            img[key].update({'img': dicomImg[i].pixel_array})
            img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
            img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})
        except ValueError:
            print('MY_ERROR: key not found:\n', key)

    print(len(img), 'DICOM images extracted')
    # print(img)
    # Crop the images to given ROI
    toDelete = [] # Keep track of error causing keys and delete after loop
    for key in img:
        try:
            tmp = img[key]['img']
            x = img[key]['x']
            y = img[key]['y']
            x = [int(x[0]), int(x[1])]
            y = [int(y[0]), int(y[1])]
            #x is width, y is height
            #in numpy array, y,x
            img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
        except:
            print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
            toDelete.append(key)

    #img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
    for _ in toDelete:
        img.pop(_)
    return img


# Write images to disk with markers and basic crop    
def writeMarkedImages(img): 
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
def computeCrops(img):
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-CROP_SIZE/2+pad):int(c[1]+CROP_SIZE/2+pad), int(c[0]-CROP_SIZE/2+pad):int(c[0]+CROP_SIZE/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    return img
    
# Find bit depth
def findBitDepth(img):
    print('Find bit depth...')
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
def findAverageROISize(img):
    print('Find average ROI size...')
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
def writeCropsToDisk(img):
    count = 0
    for key in img:
        count+=1
        f = open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_crop/' + key + '.png', 'wb')
        w = png.Writer(width = CROP_SIZE, height = CROP_SIZE, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break
            
def buildArrayForPickle(img):    #img.update({key:{}})
    allCrops = {}
    for key in img:
        allCrops.update({key:[]})
        allCrops[key] = img[key]['crop']
    return allCrops
        
def savePickle(ob):
    import pickle
    print('Pickling...')
    with open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_normalCrop400.pickle', 'wb') as output:
        pickle.dump(ob, output, pickle.HIGHEST_PROTOCOL)
        
def buildDictNormals(dicomImg, fileList):
    from scipy import ndimage
    # Get pixel values
    # Draw square around breast
    # Select random centre for crop within breast
    img = {}
    for i in range(len(fileList)):
        key = os.path.basename(fileList[i])[:-4]
        img.update({key:{}})
        img[key].update({'img': dicomImg[i].pixel_array})
        # Get centre of mass (breast centre)
        centre = ndimage.measurements.center_of_mass(img[key]['img'])
        centre = np.asarray(centre).astype(int)
        
#         #Check images
#         plt.figure(figsize=(20,20))  
#         #plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
#         plt.imshow(img[key]['img'], cmap='gray')
#         plt.plot(centre[1], centre[0], marker='x', color=[1,0,1], markersize=30)
#         #plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_full.png')
#         plt.show()
#         plt.close()

        #LATER
#         img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
#         img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})


print('Done')

Started...
Done


2692


In [20]:
# Load images into ram
dicomImg, fileList = getFiles()    
print('Done')

3248  dicom images found
3248  Files found
Loading images...
1 / 3248
2 / 3248
3 / 3248
4 / 3248
5 / 3248
6 / 3248
7 / 3248
8 / 3248
9 / 3248
10 / 3248
11 / 3248
12 / 3248
13 / 3248
14 / 3248
15 / 3248
16 / 3248
17 / 3248
18 / 3248
19 / 3248
20 / 3248
21 / 3248
22 / 3248
23 / 3248
24 / 3248
25 / 3248
26 / 3248
27 / 3248
28 / 3248
29 / 3248
30 / 3248
31 / 3248
32 / 3248
33 / 3248
34 / 3248
35 / 3248
36 / 3248
37 / 3248
38 / 3248
39 / 3248
40 / 3248
41 / 3248
42 / 3248
43 / 3248
44 / 3248
45 / 3248
46 / 3248
47 / 3248
48 / 3248
49 / 3248
50 / 3248
51 / 3248
52 / 3248
53 / 3248
54 / 3248
55 / 3248
56 / 3248
57 / 3248
58 / 3248
59 / 3248
60 / 3248
61 / 3248
62 / 3248
63 / 3248
64 / 3248
65 / 3248
66 / 3248
67 / 3248
68 / 3248
69 / 3248
70 / 3248
71 / 3248
72 / 3248
73 / 3248
74 / 3248
75 / 3248
76 / 3248
77 / 3248
78 / 3248
79 / 3248
80 / 3248
81 / 3248
82 / 3248
83 / 3248
84 / 3248
85 / 3248
86 / 3248
87 / 3248
88 / 3248
89 / 3248
90 / 3248
91 / 3248
92 / 3248
93 / 3248
94 / 3248
95 / 324

751 / 3248
752 / 3248
753 / 3248
754 / 3248
755 / 3248
756 / 3248
757 / 3248
758 / 3248
759 / 3248
760 / 3248
761 / 3248
762 / 3248
763 / 3248
764 / 3248
765 / 3248
766 / 3248
767 / 3248
768 / 3248
769 / 3248
770 / 3248
771 / 3248
772 / 3248
773 / 3248
774 / 3248
775 / 3248
776 / 3248
777 / 3248
778 / 3248
779 / 3248
780 / 3248
781 / 3248
782 / 3248
783 / 3248
784 / 3248
785 / 3248
786 / 3248
787 / 3248
788 / 3248
789 / 3248
790 / 3248
791 / 3248
792 / 3248
793 / 3248
794 / 3248
795 / 3248
796 / 3248
797 / 3248
798 / 3248
799 / 3248
800 / 3248
801 / 3248
802 / 3248
803 / 3248
804 / 3248
805 / 3248
806 / 3248
807 / 3248
808 / 3248
809 / 3248
810 / 3248
811 / 3248
812 / 3248
813 / 3248
814 / 3248
815 / 3248
816 / 3248
817 / 3248
818 / 3248
819 / 3248
820 / 3248
821 / 3248
822 / 3248
823 / 3248
824 / 3248
825 / 3248
826 / 3248
827 / 3248
828 / 3248
829 / 3248
830 / 3248
831 / 3248
832 / 3248
833 / 3248
834 / 3248
835 / 3248
836 / 3248
837 / 3248
838 / 3248
839 / 3248
840 / 3248
841 / 3248

1455 / 3248
1456 / 3248
1457 / 3248
1458 / 3248
1459 / 3248
1460 / 3248
1461 / 3248
1462 / 3248
1463 / 3248
1464 / 3248
1465 / 3248
1466 / 3248
1467 / 3248
1468 / 3248
1469 / 3248
1470 / 3248
1471 / 3248
1472 / 3248
1473 / 3248
1474 / 3248
1475 / 3248
1476 / 3248
1477 / 3248
1478 / 3248
1479 / 3248
1480 / 3248
1481 / 3248
1482 / 3248
1483 / 3248
1484 / 3248
1485 / 3248
1486 / 3248
1487 / 3248
1488 / 3248
1489 / 3248
1490 / 3248
1491 / 3248
1492 / 3248
1493 / 3248
1494 / 3248
1495 / 3248
1496 / 3248
1497 / 3248
1498 / 3248
1499 / 3248
1500 / 3248
1501 / 3248
1502 / 3248
1503 / 3248
1504 / 3248
1505 / 3248
1506 / 3248
1507 / 3248
1508 / 3248
1509 / 3248
1510 / 3248
1511 / 3248
1512 / 3248
1513 / 3248
1514 / 3248
1515 / 3248
1516 / 3248
1517 / 3248
1518 / 3248
1519 / 3248
1520 / 3248
1521 / 3248
1522 / 3248
1523 / 3248
1524 / 3248
1525 / 3248
1526 / 3248
1527 / 3248
1528 / 3248
1529 / 3248
1530 / 3248
1531 / 3248
1532 / 3248
1533 / 3248
1534 / 3248
1535 / 3248
1536 / 3248
1537 / 3248
1538

2138 / 3248
2139 / 3248
2140 / 3248
2141 / 3248
2142 / 3248
2143 / 3248
2144 / 3248
2145 / 3248
2146 / 3248
2147 / 3248
2148 / 3248
2149 / 3248
2150 / 3248
2151 / 3248
2152 / 3248
2153 / 3248
2154 / 3248
2155 / 3248
2156 / 3248
2157 / 3248
2158 / 3248
2159 / 3248
2160 / 3248
2161 / 3248
2162 / 3248
2163 / 3248
2164 / 3248
2165 / 3248
2166 / 3248
2167 / 3248
2168 / 3248
2169 / 3248
2170 / 3248
2171 / 3248
2172 / 3248
2173 / 3248
2174 / 3248
2175 / 3248
2176 / 3248
2177 / 3248
2178 / 3248
2179 / 3248
2180 / 3248
2181 / 3248
2182 / 3248
2183 / 3248
2184 / 3248
2185 / 3248
2186 / 3248
2187 / 3248
2188 / 3248
2189 / 3248
2190 / 3248
2191 / 3248
2192 / 3248
2193 / 3248
2194 / 3248
2195 / 3248
2196 / 3248
2197 / 3248
2198 / 3248
2199 / 3248
2200 / 3248
2201 / 3248
2202 / 3248
2203 / 3248
2204 / 3248
2205 / 3248
2206 / 3248
2207 / 3248
2208 / 3248
2209 / 3248
2210 / 3248
2211 / 3248
2212 / 3248
2213 / 3248
2214 / 3248
2215 / 3248
2216 / 3248
2217 / 3248
2218 / 3248
2219 / 3248
2220 / 3248
2221

2821 / 3248
2822 / 3248
2823 / 3248
2824 / 3248
2825 / 3248
2826 / 3248
2827 / 3248
2828 / 3248
2829 / 3248
2830 / 3248
2831 / 3248
2832 / 3248
2833 / 3248
2834 / 3248
2835 / 3248
2836 / 3248
2837 / 3248
2838 / 3248
2839 / 3248
2840 / 3248
2841 / 3248
2842 / 3248
2843 / 3248
2844 / 3248
2845 / 3248
2846 / 3248
2847 / 3248
2848 / 3248
2849 / 3248
2850 / 3248
2851 / 3248
2852 / 3248
2853 / 3248
2854 / 3248
2855 / 3248
2856 / 3248
2857 / 3248
2858 / 3248
2859 / 3248
2860 / 3248
2861 / 3248
2862 / 3248
2863 / 3248
2864 / 3248
2865 / 3248
2866 / 3248
2867 / 3248
2868 / 3248
2869 / 3248
2870 / 3248
2871 / 3248
2872 / 3248
2873 / 3248
2874 / 3248
2875 / 3248
2876 / 3248
2877 / 3248
2878 / 3248
2879 / 3248
2880 / 3248
2881 / 3248
2882 / 3248
2883 / 3248
2884 / 3248
2885 / 3248
2886 / 3248
2887 / 3248
2888 / 3248
2889 / 3248
2890 / 3248
2891 / 3248
2892 / 3248
2893 / 3248
2894 / 3248
2895 / 3248
2896 / 3248
2897 / 3248
2898 / 3248
2899 / 3248
2900 / 3248
2901 / 3248
2902 / 3248
2903 / 3248
2904

In [ ]:
# Get list of files that are for presentation and have ROIs
# Copy these files to a new folder
# Get the contralateral images 
# Copy contralateral images to a new folder
from shutil import copyfile
dstCopy = '/vol/research/mammo2/will/data/batches/roi/batch_3/lesions'
#Get list of files that are in the spreadsheet with ROIs
fileListROI = getFileListROI(fileList)
#Copy files to new folder
for index, path in enumerate(fileListROI):
    copyfile(path, dstCopy + '/' + os.path.basename(fileList[index]))
    print(index, '\\', len(fileListROI) )


MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.3609.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.3608.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.3615.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.3620.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.3621.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.3627.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.6216.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.6210.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.6203.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.6215.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.6204.0

 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4073.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4078.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4106.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4086.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4102.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4076.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4096.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4094.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4083.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4104.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.4088.0
MY_ERROR: key not found:

 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.2662.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.2661.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.2670.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.2673.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.2674.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.2666.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1728.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1749.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1727.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1736.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1744.0
MY_ERROR: key not found:

 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1101.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1127.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1112.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1109.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1097.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1114.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1125.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1077.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1083.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1065.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1388821.1077.1517512309538.1071.0
MY_ERROR: key not found:

0 \ 400
1 \ 400
2 \ 400
3 \ 400
4 \ 400
5 \ 400
6 \ 400
7 \ 400
8 \ 400
9 \ 400
10 \ 400
11 \ 400
12 \ 400
13 \ 400
14 \ 400
15 \ 400
16 \ 400
17 \ 400
18 \ 400
19 \ 400
20 \ 400
21 \ 400
22 \ 400
23 \ 400
24 \ 400
25 \ 400
26 \ 400
27 \ 400
28 \ 400
29 \ 400
30 \ 400
31 \ 400
32 \ 400
33 \ 400
34 \ 400
35 \ 400
36 \ 400
37 \ 400
38 \ 400
39 \ 400
40 \ 400
41 \ 400
42 \ 400
43 \ 400
44 \ 400
45 \ 400
46 \ 400
47 \ 400
48 \ 400
49 \ 400
50 \ 400
51 \ 400
52 \ 400
53 \ 400
54 \ 400
55 \ 400
56 \ 400
57 \ 400
58 \ 400
59 \ 400
60 \ 400
61 \ 400
62 \ 400
63 \ 400
64 \ 400
65 \ 400
66 \ 400
67 \ 400
68 \ 400
69 \ 400
70 \ 400
71 \ 400
72 \ 400
73 \ 400
74 \ 400
75 \ 400
76 \ 400
77 \ 400
78 \ 400
79 \ 400
80 \ 400
81 \ 400
82 \ 400
83 \ 400
84 \ 400
85 \ 400
86 \ 400
87 \ 400
88 \ 400
89 \ 400
90 \ 400
91 \ 400
92 \ 400
93 \ 400
94 \ 400
95 \ 400
96 \ 400
97 \ 400
98 \ 400
99 \ 400
100 \ 400
101 \ 400
102 \ 400
103 \ 400
104 \ 400
105 \ 400
106 \ 400
107 \ 400
108 \ 400
109 \ 400
110 \ 400


In [18]:
dicomImg, fileList = deletePreProcessed(dicomImg, fileList)
dicomImg = deletePreProcessed(dicomImg, fileList)
img = buildDict(dicomImg, fileList)
img = computeCrops(img)
#buildDictNormals(dicomImg, fileList)
# findAverageROISize(img)
# findBitDepth(img)
# writeMarkedImages(img)
# writeCropsToDisk(img)
# savePickle(buildArrayForPickle(img)) 

AttributeError: 'numpy.ndarray' object has no attribute 'PresentationIntentType'

In [ ]:
# Copy contralateral image


In [41]:
# Read the pickle file
import pickle
objects = []
with (open("/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_malignantCrop256.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
# for key in image:
#     print(key)
#print(objects[0]['crop'].shape)
images = objects[0]

print('Find bit depth...')
maxmax = 0
for key in images:
    tmp = images[key]
    print(np.amax(tmp))
    if np.amax(tmp) > maxmax:
        maxmax = np.amax(tmp)
print('The largest value is: ', maxmax)




Find bit depth...
858
724
735
566
760
13717
657
618
511
760
2129
725
662
674
483
11723
772
566
1267
897
684
751
983
679
654
1556
861
565
729
673
679
788
616
7089
555
502
822
705
661
758
632
749
650
490
677
11926
11483
2129
704
668
665
1004
770
603
1051
682
1236
584
665
1050
654
830
1073
781
773
661
734
735
1319
724
797
729
665
998
744
781
9982
552
930
777
697
679
14222
773
763
952
631
773
658
791
532
747
1071
720
807
717
657
688
741
16383
729
780
617
772
536
522
533
618
674
982
607
798
798
694
694
742
731
1135
995
700
858
836
575
707
894
680
1075
722
1269
1115
668
732
753
563
865
1941
1400
661
768
772
1133
1154
697
509
615
16383
698
705
8948
616
867
864
2951
665
701
713
788
4739
875
684
884
7484
522
1137
1102
541
1113
740
617
1413
597
613
860
942
545
637
511
656
722
587
674
726
5446
685
816
1954
741
769
654
608
828
760
794
15812
2071
680
833
624
1211
704
572
604
606
582
769
599
718
535
577
594
905
502
569
504
749
655
584
785
16383
555
790
659
671
817
648
10970
763
752
12259
909
6188
75